# 範例 10：查看 LM Studio 中可用的模型

這個範例展示如何取得 LM Studio 目前載入的模型清單。

## 學習目標
- 學會使用 Models API
- 了解如何動態取得可用模型
- 建立更靈活的應用程式

## 為什麼需要查詢模型？
- 確認模型是否已載入
- 動態選擇要使用的模型
- 建立支援多模型的應用程式

## 前置需求
- LM Studio 運行中，Local Server 已啟動

## Step 1: 匯入套件

In [ ]:
import requests

## Step 2: 定義查詢模型的函數

In [ ]:
def list_models():
    """
    取得 LM Studio 中可用的模型清單
    """

    url = "http://localhost:1234/v1/models"

    response = requests.get(url)
    result = response.json()

    print("=== LM Studio 可用模型 ===")
    print("-" * 40)

    for model in result["data"]:
        print(f"• {model['id']}")

    return result["data"]

## Step 3: 查詢可用模型

In [ ]:
models = list_models()
print(f"\n共 {len(models)} 個模型可用")

## Step 4: 查看模型詳細資訊

In [ ]:
def get_model_details():
    """
    取得模型的詳細資訊
    """
    url = "http://localhost:1234/v1/models"
    response = requests.get(url)
    result = response.json()
    
    print("=== 模型詳細資訊 ===")
    for model in result["data"]:
        print(f"\n模型 ID: {model['id']}")
        print(f"物件類型: {model.get('object', 'N/A')}")
        print(f"擁有者: {model.get('owned_by', 'N/A')}")
        if 'created' in model:
            print(f"建立時間: {model['created']}")
        print("-" * 30)
    
    return result["data"]

details = get_model_details()

## Step 5: 使用 OpenAI SDK 查詢模型

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

# 使用 SDK 取得模型清單
models = client.models.list()

print("=== 使用 OpenAI SDK 取得的模型 ===")
for model in models.data:
    print(f"• {model.id}")

## Step 6: 動態選擇模型

In [ ]:
def chat_with_first_available_model(message):
    """
    自動使用第一個可用的模型進行對話
    """
    # 取得可用模型
    models = client.models.list()
    
    if not models.data:
        return "沒有可用的模型！請在 LM Studio 中載入模型。"
    
    # 使用第一個模型
    model_id = models.data[0].id
    print(f"使用模型：{model_id}")
    
    # 發送請求
    response = client.chat.completions.create(
        model=model_id,
        messages=[{"role": "user", "content": message}]
    )
    
    return response.choices[0].message.content

In [ ]:
# 測試動態選擇模型
print("\n=== 動態選擇模型測試 ===")
answer = chat_with_first_available_model("什麼是 Python？")
print(f"\n回答：{answer}")

## Step 7: 檢查特定模型是否可用

In [ ]:
def is_model_available(model_name):
    """
    檢查指定的模型是否可用
    
    參數：
        model_name: 模型名稱
    
    回傳：
        True 如果模型可用，否則 False
    """
    models = client.models.list()
    available_ids = [m.id for m in models.data]
    return model_name in available_ids

# 測試
test_model = "gpt-oss-120b"
if is_model_available(test_model):
    print(f"✓ 模型 '{test_model}' 可用")
else:
    print(f"✗ 模型 '{test_model}' 不可用")
    
# 測試一個不存在的模型
fake_model = "not-a-real-model"
if is_model_available(fake_model):
    print(f"✓ 模型 '{fake_model}' 可用")
else:
    print(f"✗ 模型 '{fake_model}' 不可用")

## API 端點說明

| 端點 | 方法 | 說明 |
|------|------|------|
| `/v1/models` | GET | 取得所有可用模型清單 |
| `/v1/models/{model_id}` | GET | 取得特定模型資訊 |

### 回應格式
```json
{
    "data": [
        {
            "id": "model-name",
            "object": "model",
            "owned_by": "organization"
        }
    ]
}
```

## 重點回顧

1. **Models API**：使用 `/v1/models` 取得模型清單
2. **動態選擇**：可以根據可用模型動態調整程式
3. **錯誤處理**：在使用模型前先確認它是否可用
4. **OpenAI SDK**：可以使用 `client.models.list()` 取得模型

## 下一步

在下一個範例中，我們將建立一個可以在 Ollama 和 LM Studio 之間切換的通用聊天程式！